# 8.2 Training a convnet from scratch on a small dataset



In [4]:
!pip install kaggle

In [2]:
!cp /Users/ronaldnetawat/Desktop/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats

100%|███████████████████████████████████████▊| 809M/812M [00:19<00:00, 40.3MB/s]
100%|████████████████████████████████████████| 812M/812M [00:19<00:00, 43.1MB/s]


In [6]:
!unzip -qq dogs-vs-cats.zip

In [7]:
!unzip -qq train.zip

### Copying images to train, validation and test directories

In [2]:
import os, shutil, pathlib 

In [3]:
original_dir = pathlib.Path("train") #path to directory where original dataset was uncompressed
new_base_dir = pathlib.Path("cats_vs_dogs_small") # directory to store our smaller dataset

In [4]:
# utility func to copy cat/dog images from start_index to end_index to the subdir
# new_base_dir/{subset_name}/cat(and /dog)
# subset_name options: "train", "validation", "test"
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

make_subset("train", start_index=0, end_index=1000) #create training subset w/ first 1000 images of each category
make_subset("validation", start_index=1000, end_index=1500) #create validation subset w/ 500 images of each category
make_subset("test", start_index=1500, end_index=2500) # create test subset w/ 1000 images of each category

FileExistsError: [Errno 17] File exists: 'cats_vs_dogs_small/train/cat'

### Building the model

- Same structure as the example done before, just adding 2 more `Conv2D` and `MaxPooling2D` layers.
- Size of feature maps will go from 180x180 pixels to 7 x 7.
- Last layer will be a `Dense` layer with 1 unit and `sigmoid` activation for binary classification.
- Also, we'll start the model with a `Rescaling` layer to rescale image inputs from [0,255] range to [0,1] range.

In [7]:
import tensorflow as tf

NotFoundError: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <2814A58E-D752-317B-8040-131217E2F9AA> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

NotFoundError: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <2814A58E-D752-317B-8040-131217E2F9AA> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [12]:
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [13]:
# model summary

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        12,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 991,041 (3.78 MB)

 Trainable params: 991,041 (3.78 MB)

 Non-trainable params: 0 (0.00 B)

### Compiling: use `RMSProp` optimizer and binary crossentropy as loss

In [14]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

Turning the data into the appropriate data type: floating-point tensors. Steps:

1. Read the picture files.
2. Decode the JPEG content to RGB grids of pixels.
3. Convert them into floating-point tensors.
4. Resize them to 180 x 180.
5. Pack them into batches (batch size of 32).

Use `image_dataset_from_directory()` utility function from keras to set up a data pipeline that can automatically turn image files on disk into batches of preprocessed tensors.

In [15]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

# this returns a tf.data.Dataset object configured to read these files, shuffle them, decode them to tensors, resize them to a shared size, and pack them into batches.

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [16]:
# shapes of the data and labele yielded by the Dataset

for data_batch, labels_batch in train_dataset:
    print("data batch shape: ", data_batch.shape)
    print("labels batch shape: ", labels_batch.shape)
    break

data batch shape:  (32, 180, 180, 3)
labels batch shape:  (32,)


### Fitting the model using `Dataset`

- Use `validation_data` argument to monitor validation metrics.
- Use `ModelCheckpoint` callback to save model after each epoch.
    - `save_best_only=True` and `monitor="val_loss"`

In [17]:
callbacks = [keras.callbacks.ModelCheckpoint(
    filepath="convnet_from_scratch.keras",
    save_best_only=True,
    monitor="val_loss")]

In [20]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 36s 575ms/step - accuracy: 0.8390 - loss: 0.3575 - val_accuracy: 0.6830 - val_loss: 0.6257
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 529ms/step - accuracy: 0.8691 - loss: 0.3125 - val_accuracy: 0.7460 - val_loss: 0.5661
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 530ms/step - accuracy: 0.9015 - loss: 0.2442 - val_accuracy: 0.6980 - val_loss: 0.8697
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 36s 569ms/step - accuracy: 0.9116 - loss: 0.1966 - val_accuracy: 0.7310 - val_loss: 0.7927
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 32s 502ms/step - accuracy: 0.9404 - loss: 0.1393 - val_accuracy: 0.7310 - val_loss: 0.8172
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.9629 - loss: 0.1112

KeyboardInterrupt: 

In [ ]:
# plotting loss curves and accuracy during training

import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(accuracy) + 1)

plt.plot(epochs, accuracy, "bo", label="training accuracy")
plt.plot(epochs, val_accuracy, "b", label="validation accuracy")
plt.title("training and validation accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, loss, "bo", label="training loss")
plt.plot(epochs, val_loss, "b", label="validation loss")
plt.title("training and validation loss")
plt.legend()
plt.show()

In [2]:
import tensorflow as tf

print("tf devices: ", tf.config.list_physical_devices())

print("\ngpu available: ", tf.config.list_physical_devices('GPU'))

tf.debugging.set_log_device_placement(True)

NotFoundError: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <2814A58E-D752-317B-8040-131217E2F9AA> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so